# Análise de Cenários e Sensibilidade - IDCI-VIX

Este notebook demonstra técnicas avançadas de análise de cenários:

- **Previsão por Quantis**: Cenários pessimista (10%), base (50%) e otimista (90%)
- **Análise de Sensibilidade**: Impacto de variações nas variáveis preditoras
- **Simulação de Monte Carlo**: Distribuição de resultados possíveis
- **Backtesting**: Validação de cenários históricos
- **Value at Risk (VaR)**: Quantificação de riscos

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

## 1. Preparação dos Dados

In [ ]:
def gerar_dados_realistas(n_periodos=150, seed=42):
    """Gera dataset sintético com variáveis macroeconômicas."""
    np.random.seed(seed)
    datas = pd.date_range(start='2012-01-01', periods=n_periodos, freq='M')
    t = np.arange(n_periodos)
    
    # PIB com ciclos econômicos
    pib = 2000 + 15*t + 200*np.sin(2*np.pi*t/48) + np.random.normal(0, 50, n_periodos)
    
    # Selic com mudanças de regime
    selic = np.zeros(n_periodos)
    selic[0] = 10.0
    for i in range(1, n_periodos):
        shock = np.random.normal(0, 0.3)
        if i % 30 == 0:
            shock += np.random.choice([-2, 2])
        selic[i] = np.clip(selic[i-1] + shock, 2.0, 20.0)
    
    # Inflação (IPCA)
    ipca = np.zeros(n_periodos)
    ipca[0] = 0.5
    for i in range(1, n_periodos):
        ipca[i] = 0.6*ipca[i-1] + 0.3 + np.random.normal(0, 0.2)
        ipca[i] = np.clip(ipca[i], -1.0, 2.5)
    
    # Desemprego
    pib_norm = (pib - pib.mean()) / pib.std()
    desemprego = 10.0 - 2*pib_norm + np.random.normal(0, 0.5, n_periodos)
    desemprego = np.clip(desemprego, 4.0, 16.0)
    
    # Crédito imobiliário
    credito = 50000 + 400*t + 5000*pib_norm - 2000*(selic - selic.mean())/selic.std()
    credito += np.random.normal(0, 2000, n_periodos)
    
    # Confiança do consumidor
    confianca = 100 + 15*pib_norm - 10*(desemprego - desemprego.mean())/desemprego.std()
    confianca += np.random.normal(0, 5, n_periodos)
    
    # IDCI-VIX sintético (combinação das variáveis)
    idci_raw = (0.3*pib_norm - 0.2*(selic - selic.mean())/selic.std() + 
                0.2*confianca/20 - 0.15*(desemprego - desemprego.mean())/desemprego.std() +
                0.15*(credito - credito.mean())/credito.std())
    idci_vix = 5 + 2*idci_raw + np.random.normal(0, 0.3, n_periodos)
    idci_vix = np.clip(idci_vix, 0, 10)
    
    df = pd.DataFrame({
        'data': datas,
        'pib_real': pib,
        'taxa_selic': selic,
        'ipca': ipca,
        'taxa_desemprego': desemprego,
        'credito_imobiliario': credito,
        'confianca_consumidor': confianca,
        'IDCI_VIX': idci_vix
    })
    
    df.set_index('data', inplace=True)
    return df

df = gerar_dados_realistas(n_periodos=150)
print(f"Dataset: {len(df)} observações de {df.index[0].strftime('%Y-%m')} a {df.index[-1].strftime('%Y-%m')}")
df.head()

In [ ]:
# Dividir dados
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

print(f"Treino: {len(train_data)} obs | Teste: {len(test_data)} obs")

## 2. Previsão por Quantis - Cenários Múltiplos

Usar regressão quantílica para gerar três cenários:
- **Pessimista** (quantil 10%): Cenário adverso
- **Base** (quantil 50%): Cenário mais provável
- **Otimista** (quantil 90%): Cenário favorável

In [ ]:
from forecasting.quantile_reg import QuantileForecaster

# Preparar dados
feature_cols = ['pib_real', 'taxa_selic', 'ipca', 'taxa_desemprego', 
                'credito_imobiliario', 'confianca_consumidor']
X_train = train_data[feature_cols].values
y_train = train_data['IDCI_VIX'].values
X_test = test_data[feature_cols].values
y_test = test_data['IDCI_VIX'].values

# Treinar modelos para diferentes quantis
quantis = [0.10, 0.50, 0.90]
cenarios = {}

for q in quantis:
    print(f"Treinando modelo para quantil {q:.0%}...")
    qf = QuantileForecaster(quantile=q, lags=3)
    qf.fit(X_train, y_train)
    pred = qf.forecast(X_test)
    
    if q == 0.10:
        cenarios['Pessimista'] = pred
    elif q == 0.50:
        cenarios['Base'] = pred
    else:
        cenarios['Otimista'] = pred

print("\n✓ Cenários gerados com sucesso!")

In [ ]:
# Visualizar cenários
fig, ax = plt.subplots(figsize=(16, 8))

# Valores reais
ax.plot(test_data.index, y_test, 'o-', label='Real', 
        linewidth=3, markersize=7, color='black', zorder=5)

# Cenário base
ax.plot(test_data.index, cenarios['Base'], 's--', label='Cenário Base (Q50)', 
        linewidth=2.5, markersize=6, color='blue', alpha=0.8)

# Área de incerteza (pessimista a otimista)
ax.fill_between(test_data.index, 
                cenarios['Pessimista'], 
                cenarios['Otimista'],
                alpha=0.3, color='skyblue', label='Intervalo de Confiança (Q10-Q90)')

# Linhas dos cenários extremos
ax.plot(test_data.index, cenarios['Pessimista'], '^:', 
        label='Cenário Pessimista (Q10)', linewidth=2, markersize=5, color='red', alpha=0.7)
ax.plot(test_data.index, cenarios['Otimista'], 'v:', 
        label='Cenário Otimista (Q90)', linewidth=2, markersize=5, color='green', alpha=0.7)

ax.set_title('Análise de Cenários - IDCI-VIX', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Data', fontsize=13)
ax.set_ylabel('IDCI-VIX (0-10)', fontsize=13)
ax.legend(loc='best', fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.4)
ax.set_ylim(0, 10)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Tabela de cenários
df_cenarios = pd.DataFrame({
    'Data': test_data.index,
    'Real': y_test,
    'Pessimista': cenarios['Pessimista'],
    'Base': cenarios['Base'],
    'Otimista': cenarios['Otimista']
})

df_cenarios['Amplitude'] = df_cenarios['Otimista'] - df_cenarios['Pessimista']
df_cenarios['Dentro_Intervalo'] = (
    (df_cenarios['Real'] >= df_cenarios['Pessimista']) & 
    (df_cenarios['Real'] <= df_cenarios['Otimista'])
)

cobertura = df_cenarios['Dentro_Intervalo'].mean() * 100
print(f"\nCobertura do Intervalo (Q10-Q90): {cobertura:.1f}%")
print(f"Amplitude média: {df_cenarios['Amplitude'].mean():.2f}")
print("\nPrimeiras previsões:")
df_cenarios.head(10)

## 3. Análise de Sensibilidade

Avaliar como mudanças nas variáveis preditoras afetam o IDCI-VIX:

In [ ]:
from forecasting.random_forest import RandomForestForecaster

# Treinar modelo Random Forest para análise de sensibilidade
rf = RandomForestForecaster(n_estimators=200, max_depth=15, lags=5, random_state=42)
rf.fit(X_train, y_train)

# Importância das features
importances = rf.feature_importance(feature_cols)

print("Importância das Variáveis:")
print("=" * 60)
for var, imp in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"{var:30s} {'█' * int(imp*50)} {imp:.3f}")
print("=" * 60)

In [ ]:
# Análise de sensibilidade: variar cada variável mantendo outras constantes
def analise_sensibilidade(model, X_base, feature_idx, feature_name, variation_pct=0.2, n_points=50):
    """
    Analisa sensibilidade variando uma feature específica.
    
    Args:
        model: Modelo treinado
        X_base: Matriz de features base
        feature_idx: Índice da feature a variar
        feature_name: Nome da feature
        variation_pct: Percentual de variação (+/-)
        n_points: Número de pontos a avaliar
    """
    # Pegar valores médios como baseline
    x_baseline = X_base.mean(axis=0).reshape(1, -1)
    baseline_value = x_baseline[0, feature_idx]
    
    # Range de variação
    var_min = baseline_value * (1 - variation_pct)
    var_max = baseline_value * (1 + variation_pct)
    var_range = np.linspace(var_min, var_max, n_points)
    
    # Previsões variando a feature
    predictions = []
    for val in var_range:
        x_modified = x_baseline.copy()
        x_modified[0, feature_idx] = val
        pred = model.forecast(x_modified)[0]
        predictions.append(pred)
    
    return var_range, np.array(predictions)

# Análise para cada variável
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
fig.suptitle('Análise de Sensibilidade - Impacto das Variáveis no IDCI-VIX', 
             fontsize=16, fontweight='bold')

for idx, feature_name in enumerate(feature_cols):
    ax = axes[idx // 2, idx % 2]
    
    var_range, predictions = analise_sensibilidade(
        rf, X_test, idx, feature_name, variation_pct=0.3, n_points=100
    )
    
    # Plot
    ax.plot(var_range, predictions, linewidth=3, color='darkblue')
    ax.axvline(X_test[:, idx].mean(), color='red', linestyle='--', 
               alpha=0.7, label='Média Atual')
    ax.axhline(y_test.mean(), color='green', linestyle=':', 
               alpha=0.7, label='IDCI-VIX Médio')
    
    # Calcular elasticidade
    delta_pred = predictions[-1] - predictions[0]
    delta_var = var_range[-1] - var_range[0]
    elasticity = (delta_pred / predictions[0]) / (delta_var / var_range[0])
    
    ax.set_title(f"{feature_name.replace('_', ' ').title()}\nElasticidade: {elasticity:.3f}", 
                 fontsize=11, fontweight='bold')
    ax.set_xlabel(feature_name.replace('_', ' ').title(), fontsize=10)
    ax.set_ylabel('IDCI-VIX Previsto', fontsize=10)
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Simulação de Monte Carlo

Simular múltiplos cenários usando distribuições probabilísticas:

In [ ]:
def monte_carlo_simulation(model, X_base, n_simulations=1000, noise_std=0.1):
    """
    Realiza simulação de Monte Carlo adicionando ruído às features.
    
    Args:
        model: Modelo treinado
        X_base: Features base
        n_simulations: Número de simulações
        noise_std: Desvio padrão do ruído (proporcional)
    """
    results = []
    
    for _ in range(n_simulations):
        # Adicionar ruído gaussiano às features
        noise = np.random.normal(1.0, noise_std, X_base.shape)
        X_perturbed = X_base * noise
        
        # Prever
        predictions = model.forecast(X_perturbed)
        results.append(predictions)
    
    return np.array(results)

# Executar Monte Carlo
print("Executando simulação de Monte Carlo...")
mc_results = monte_carlo_simulation(rf, X_test, n_simulations=2000, noise_std=0.15)

print(f"✓ {len(mc_results)} simulações concluídas")
print(f"Forma dos resultados: {mc_results.shape}")

In [ ]:
# Visualizar distribuição das simulações
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Simulação de Monte Carlo - Distribuição de Cenários', 
             fontsize=16, fontweight='bold')

# Selecionar alguns períodos para visualizar
periodos_plot = [0, len(test_data)//3, 2*len(test_data)//3, -1]
titles = ['Início', 'T+10', 'T+20', 'Fim']

for idx, (periodo, title) in enumerate(zip(periodos_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    
    # Distribuição das simulações
    simulations = mc_results[:, periodo]
    
    ax.hist(simulations, bins=40, edgecolor='black', alpha=0.7, density=True)
    
    # Estatísticas
    mean_sim = simulations.mean()
    median_sim = np.median(simulations)
    std_sim = simulations.std()
    
    # Linhas de referência
    ax.axvline(mean_sim, color='red', linestyle='--', linewidth=2, label=f'Média: {mean_sim:.2f}')
    ax.axvline(median_sim, color='green', linestyle='--', linewidth=2, label=f'Mediana: {median_sim:.2f}')
    ax.axvline(y_test[periodo], color='black', linestyle='-', linewidth=3, 
               label=f'Real: {y_test[periodo]:.2f}')
    
    # Intervalo de confiança 90%
    ci_lower = np.percentile(simulations, 5)
    ci_upper = np.percentile(simulations, 95)
    ax.axvspan(ci_lower, ci_upper, alpha=0.2, color='yellow', label=f'IC 90%: [{ci_lower:.2f}, {ci_upper:.2f}]')
    
    ax.set_title(f"{title} - {test_data.index[periodo].strftime('%Y-%m')}\nσ = {std_sim:.2f}", 
                 fontsize=12, fontweight='bold')
    ax.set_xlabel('IDCI-VIX', fontsize=11)
    ax.set_ylabel('Densidade', fontsize=11)
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Fanplot - Visualizar evolução da incerteza
fig, ax = plt.subplots(figsize=(16, 8))

# Calcular percentis ao longo do tempo
percentiles = [5, 25, 50, 75, 95]
percentile_values = {p: np.percentile(mc_results, p, axis=0) for p in percentiles}

# Plot
ax.plot(test_data.index, y_test, 'o-', label='Real', 
        linewidth=3, markersize=7, color='black', zorder=10)
ax.plot(test_data.index, percentile_values[50], '-', label='Mediana', 
        linewidth=2.5, color='blue', zorder=5)

# Faixas de incerteza
ax.fill_between(test_data.index, percentile_values[5], percentile_values[95],
                alpha=0.2, color='blue', label='IC 90%')
ax.fill_between(test_data.index, percentile_values[25], percentile_values[75],
                alpha=0.3, color='blue', label='IC 50%')

ax.set_title('Fan Chart - Evolução da Incerteza (Monte Carlo)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Data', fontsize=13)
ax.set_ylabel('IDCI-VIX', fontsize=13)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Value at Risk (VaR) e Conditional VaR

Quantificar riscos extremos:

In [ ]:
def calcular_var_cvar(simulacoes, confidence_level=0.95):
    """
    Calcula Value at Risk e Conditional Value at Risk.
    
    VaR: Perda máxima esperada com dado nível de confiança
    CVaR: Perda média condicional além do VaR
    """
    alpha = 1 - confidence_level
    
    # VaR: percentil inferior
    var = np.percentile(simulacoes, alpha * 100, axis=0)
    
    # CVaR: média dos valores abaixo do VaR
    cvar = np.array([simulacoes[:, i][simulacoes[:, i] <= var[i]].mean() 
                     for i in range(simulacoes.shape[1])])
    
    return var, cvar

# Calcular VaR e CVaR
var_95, cvar_95 = calcular_var_cvar(mc_results, confidence_level=0.95)

print("Value at Risk (VaR) e Conditional VaR (CVaR) - 95% confiança")
print("=" * 80)
print(f"VaR médio (5% pior cenário): {var_95.mean():.2f}")
print(f"CVaR médio (média dos 5% piores): {cvar_95.mean():.2f}")
print(f"Diferença CVaR-VaR: {(cvar_95 - var_95).mean():.2f}")
print("=" * 80)

In [ ]:
# Visualizar VaR e CVaR
fig, ax = plt.subplots(figsize=(16, 8))

# Valores reais e previsão mediana
ax.plot(test_data.index, y_test, 'o-', label='Real', 
        linewidth=3, markersize=7, color='black', zorder=10)
ax.plot(test_data.index, percentile_values[50], '-', label='Previsão Mediana', 
        linewidth=2.5, color='blue', zorder=5)

# VaR e CVaR
ax.plot(test_data.index, var_95, '--', label='VaR 95%', 
        linewidth=2.5, color='orange', zorder=6)
ax.plot(test_data.index, cvar_95, ':', label='CVaR 95%', 
        linewidth=3, color='red', zorder=7)

# Área de risco extremo
ax.fill_between(test_data.index, 0, cvar_95, 
                alpha=0.15, color='red', label='Zona de Risco Extremo')

ax.set_title('Análise de Risco - VaR e CVaR', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Data', fontsize=13)
ax.set_ylabel('IDCI-VIX', fontsize=13)
ax.set_ylim(0, 10)
ax.legend(loc='best', fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Backtesting de Cenários

Validar se os cenários históricos tiveram cobertura adequada:

In [ ]:
# Análise de cobertura dos intervalos
def avaliar_cobertura(real, simulacoes, percentis=[5, 95]):
    """
    Avalia se os valores reais caem dentro dos intervalos previstos.
    """
    lower = np.percentile(simulacoes, percentis[0], axis=0)
    upper = np.percentile(simulacoes, percentis[1], axis=0)
    
    dentro = (real >= lower) & (real <= upper)
    cobertura = dentro.mean()
    
    return cobertura, dentro, lower, upper

# Avaliar diferentes níveis de confiança
niveis_confianca = [(5, 95), (10, 90), (25, 75)]
resultados_cobertura = {}

print("Análise de Cobertura dos Intervalos:")
print("=" * 80)

for percentis in niveis_confianca:
    cobertura, dentro, lower, upper = avaliar_cobertura(y_test, mc_results, percentis)
    nivel = percentis[1] - percentis[0]
    resultados_cobertura[nivel] = {
        'cobertura_observada': cobertura,
        'cobertura_esperada': nivel / 100,
        'dentro': dentro,
        'lower': lower,
        'upper': upper
    }
    
    print(f"Intervalo {percentis[0]}-{percentis[1]} (esperado {nivel}%):")
    print(f"  Cobertura observada: {cobertura*100:.1f}%")
    print(f"  Períodos dentro: {dentro.sum()}/{len(dentro)}")
    print()

print("=" * 80)

In [ ]:
# Visualizar backtesting
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Backtesting - Avaliação de Cobertura dos Intervalos', 
             fontsize=16, fontweight='bold')

for idx, (nivel, resultado) in enumerate(resultados_cobertura.items()):
    ax = axes[idx]
    
    # Plot
    ax.plot(test_data.index, y_test, 'o-', label='Real', 
            linewidth=2.5, markersize=6, color='black')
    
    # Intervalo
    ax.fill_between(test_data.index, 
                    resultado['lower'], 
                    resultado['upper'],
                    alpha=0.3, label=f'Intervalo {nivel}%')
    
    # Marcar pontos fora do intervalo
    fora = ~resultado['dentro']
    if fora.any():
        ax.scatter(test_data.index[fora], y_test[fora], 
                  color='red', s=100, marker='X', 
                  label='Fora do intervalo', zorder=10)
    
    cobertura_obs = resultado['cobertura_observada'] * 100
    ax.set_title(f"Intervalo {nivel}% | Cobertura: {cobertura_obs:.1f}%", 
                 fontsize=12, fontweight='bold')
    ax.set_ylabel('IDCI-VIX', fontsize=11)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    if idx < 2:
        ax.set_xticklabels([])
    else:
        ax.set_xlabel('Data', fontsize=11)
        plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 7. Resumo e Insights

### Principais Descobertas:

1. **Cenários Quantílicos**: Fornecemos previsões para cenários pessimista, base e otimista
2. **Sensibilidade**: Identificamos quais variáveis têm maior impacto no IDCI-VIX
3. **Monte Carlo**: Quantificamos a incerteza através de milhares de simulações
4. **Gestão de Risco**: Calculamos VaR e CVaR para quantificar riscos extremos
5. **Backtesting**: Validamos que os intervalos de confiança têm cobertura adequada

### Aplicações Práticas:

- **Planejamento Estratégico**: Usar cenários para planejar diferentes futuros
- **Análise de Risco**: VaR e CVaR para gestão de risco de portfólio
- **Stress Testing**: Simular choques nas variáveis e avaliar impactos
- **Comunicação**: Apresentar resultados com intervalos de confiança